### import packages

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re

### 讀取檔案
- 讀取第一頁為默認值，若檔案有多頁可加上sheet_name來指定exel頁面

In [ ]:
filename = "/Users/tim/Desktop/Python_Pandas practice projects/tp/庫存資料.xlsx"
df = pd.read_excel(filename, sheet_name="Sheet1")
df.head()

###  資料清理
■　Target　→　品名、欄位
   * 因資料頂端有頁次、日期等非必要元素，導致初始欄位不正確
   * 重新定義欄位名稱（將第4列轉為list)


In [ ]:
df.columns = list(df.iloc[4, :])
df.columns = df.columns.str.strip()
df["品名"] = df["品名"].str.strip()

■　Target → 選取需要欄位&去除Null值

In [ ]:
cols = list(df.columns)
df = df[cols[0:4] + [cols[-1]]].dropna()

■　Target → 倉庫 (留下TC1A00，捨棄期貨倉＆東興倉，但部分品項需調整)
   * 部分品項(BOPP(L)、CPP、鋁箔、PET)，需計入期貨倉，因此先進行倉庫名調整方便後續統計
   * PET12u將期貨倉合併計入，但PET藥袋23u僅算現貨倉
   * 完成調整後，僅保存TC1A00資料

In [ ]:
df.loc[(df["品名"] == "BOPP(L)"), "倉庫"] = "TC1A00"
df.loc[(df["品名"] == "CPP"), "倉庫"] = "TC1A00"
df.loc[(df["品名"].str.contains("FOIL")), "倉庫"] = "TC1A00"
df.loc[(df["品名"] == "PET") & (df["規格"].str.contains("^012")), "倉庫"] = "TC1A00"
df = df[df["倉庫"] == "TC1A00"]

■　Target → 營一部產品 (建立除去名單list，用”~“加上.isin()來過濾掉不需要的產品)
 * 完成後重新設定index，資料看起來較美觀


In [ ]:
unnecessary_list = ['X-70201S 離型劑', 'BOPP(HSD12)', 'PET(C)', "低靜電高爽滑 CPP", 'PET(H)', 'M.W.PET(I)', '矽利光離型MWPET(輕/藍/四國語言', "品名"]
df = df[~df["品名"].isin(unnecessary_list)]
df.reset_index(drop=True, inplace=True)

■　Target → 規格（從中分出厚度欄位，規格欄位僅保留長度
   * 從規格欄位分出厚度欄位 ==> 用"x"來分割，留下厚度=index[0]
   * 重新定義規格==> 用"x"來分割，留下厚度=index[1]


In [ ]:
df["厚度"] = df["規格"].str.split(pat="x").str[0].str.strip()
df["規格"] = df["規格"].str.split(pat="x").str[1].str.strip()


■　Target →  料件編號（僅保留供應商代號）
   * 重新定義料件編號 ==> 取string最後兩個字做為new value(區分供應商)
   * 注意：如果沒加.str，直接放入index會變成df["料件編號"]的index順序(e.g. df["料件編號"][18:] >>> 跳出第18筆以後的資料，在此之前為none）


In [ ]:
df["料件編號"] = df["料件編號"].str[18:]


### [品名欄位]資料變更 if necessary
 
產品分類表(供應商代號-產品)
 
 - 袋子：DY-BOPP(L) => (20 ~ 30u),(40 ~ 50u)、LA-BOPP(L) => (20 ~ 30u),(40 ~ 50u)
 - 上光：CR-BOPP(亮)、MAT OPP(霧膜)、OD-絲絨膜、預塗式-霧
 - 彩藝：CR-單面FOH (CS)]、OA-BOPP(20 ~ 40u)、OA＆DJ-PET12u、DS-藥袋PET23u、OU-CPP(20 ~ 60u)、CL＆OA-NYLON、NYLON同步、
   DB＆1X-AL、AY、DS＆OA-電鍍PET(一般、雙面電鍍)、OU-電鍍CPP25u、CR-MOP20u、DS-KOP、KPET、KNY


■　Target → 袋子

**BOPP(L)**
    
   1. 依據厚度，後面加上thin(20 ~ 25u),thick(30 ~ 50u)，再區分為寶燕or金田
   2. 注意：使用.str.contains時，如果沒有要使用正則表達法，記得將regex參數改為False(default value = True)


In [ ]:
df.loc[(df["品名"] == "BOPP(L)") & (df["料件編號"] == "LA") & (df["厚度"].isin(["020", "025"])), "品名"] = "BOPP(L)-寶燕-thin"
df.loc[(df["品名"] == "BOPP(L)") & (df["料件編號"] == "LA") & (df["厚度"].isin(["030", "040", "050"])), "品名"] = "BOPP(L)-寶燕-thick"
df.loc[(df["品名"] == "BOPP(L)") & (df["料件編號"] == "DY") & (df["厚度"].isin(["020", "025"])), "品名"] = "BOPP(L)-金田-thin"
df.loc[(df["品名"] == "BOPP(L)") & (df["料件編號"] == "DY") & (df["厚度"].isin(["030", "040", "050"])), "品名"] = "BOPP(L)-金田-thick"


■　Target → 上光

  **MAT OPP**
   * 依據厚度，區分為（上光）15u &（彩藝）20u
   * 資料中出現MAT OPP ( MTc )，視為上光品

In [ ]:
df.loc[(df["品名"].str.contains("MAT OPP", regex=False)) & (df["厚度"] == "015"), "品名"] = "MAT OPP(上光)"
df.loc[(df["品名"] == "MAT OPP") & (df["厚度"] == "020"), "品名"] = "MAT OPP(彩藝)"

■　Target → 彩藝

**PET**
  * 依據厚度，後面加上（彩藝）12u &（藥袋）19+23u，其他厚度不計入（36,50,75）

In [ ]:
df.loc[(df["品名"] == "PET") & (df["厚度"] == "012"), "品名"] = "PET(彩藝)"
df.loc[(df["品名"] == "PET") & (df["厚度"].isin(["019", "023"])), "品名"] = "PET(藥袋)"


 **NYLON**   
   * NYLON區分成NYLON(異步) & NYLON(同步)
   * 同步的NYLON FILM ( NS ) 存在一些空白格，導致計算差異，使用兩個conditions（品名包含"NS"＆料件編號為0A）來過濾

In [ ]:
df.loc[(df["品名"] == "NYLON FILM"), "品名"] = "NYLON"
df.loc[(df["品名"].str.contains("NS", regex=False)) & (df["料件編號"] == "0A"), "品名"] = "NYLON同步"

**AL**
   * 將AL FOIL & AY FOIL合併計算


In [ ]:
df.loc[(df["品名"].str.contains("FOIL",regex=False)),"品名"] = "AL"

### 新增業界欄位
- 方法 1
 * 資料清理與品名欄位調整完成後，新增業界欄位，使最終結果出來能依據業界別來分類，一目瞭然
 * 使用字典將所有品項與key綁定

In [ ]:
whole_product_type_dict ={"BOPP(L)-寶燕-thin":"袋子","BOPP(L)-寶燕-thick":"袋子","BOPP(L)-金田-thin":"袋子","BOPP(L)-金田-thick":"袋子",
                          "BOPP(亮)":"上光","MAT OPP(上光)":"上光","絲絨膜":"上光","預塗式-霧":"上光",
                          "單面FOH (CS)":"彩藝","BOPP":"彩藝","CPP":"彩藝","PET(彩藝)":"彩藝","PET(藥袋)":"彩藝","NYLON":"彩藝","NYLON同步":"彩藝","AL":"彩藝","電鍍PET":"彩藝","雙面電鍍PET":"彩藝","MCPP":"彩藝","MAT OPP(彩藝)":"彩藝","KOP":"彩藝","K-PET":"彩藝","K-NYLON FILM (N)":"彩藝"
                          }

df["業界"] = df["品名"].map(whole_product_type_dict)
print("最終資料格式表","=="*40)
print(df)

- 方法 2
 * np.select() 較為整齊


In [ ]:
condition = [df["品名"].isin(["BOPP(L)-寶燕-thin","BOPP(L)-寶燕-thick","BOPP(L)-金田-thin","BOPP(L)-金田-thick"]),
             df["品名"].isin(["BOPP(亮)","MAT OPP(上光)","絲絨膜","預塗式-霧"]),
             df["品名"].isin(["單面FOH (CS)", "BOPP", "CPP", "PET(彩藝)", "PET(藥袋)", "CPP", "NYLON","NYLON同步", "AL", "電鍍PET","雙面電鍍PET", "MCPP", "MAT OPP(彩藝)","KOP", "K-PET", "K-NYLON FILM (N)"]),
             df["品名"].isin(["補強帶( 珠光膜 )","PVC 圓點(Y)"])]

product_field = ["袋子","上光","彩藝","其他"]

df["業界"] = np.select(condition,product_field, default="Useless")
df = df.loc[df["業界"] != "Useless"]

### 統計各產品庫存量

In [ ]:
amount = round(df.groupby(["業界","品名"]).sum()["換算後庫存量"]/1000, 1)
print("各產品庫存量","=="*40)
print(amount)